In [22]:
from IPython.display import display, HTML

display(HTML("""
<style>
.output_area pre {
    white-space: pre-wrap !important;
    word-wrap: break-word !important;
}
</style>
"""))

In [7]:
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings
import requests
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv

In [8]:
load_dotenv()

True

In [6]:
%pip uninstall ddgs -y
%pip install -U duckduckgo-search

Found existing installation: ddgs 9.5.2
Uninstalling ddgs-9.5.2:
  Successfully uninstalled ddgs-9.5.2
Note: you may need to restart the kernel to use updated packages.
  Using cached duckduckgo_search-8.1.1-py3-none-any.whl.metadata (16 kB)
Using cached duckduckgo_search-8.1.1-py3-none-any.whl (18 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()

results = search_tool.invoke('top news in india today')

d:\2025_Coding\Campusx\Langchain\venv\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


In [2]:
results 

"17 hours ago · Today's top India news headlines, news on Indian politics, elections, government, business, technology, and Bollywood. 6 days ago · Get the latest breaking news from India covering politics, economy, social issues, technology, sports, and more. Stay updated … 3 days ago · Stay informed with breaking news and latest news on politics, business, entertainment sports, science, technology … 2 days ago · Read all India news and breaking news today from India on politics, business, entertainment, technology, sports, lifestyle and more at CNBC TV18. 16 hours ago · OpenAI's latest GPT-5 model boasts stronger reasoning, better coding, and improved health insights. OpenAI has unveiled …"

In [9]:
llm = ChatGroq(
    model = 'llama-3.1-8b-instant'
)

In [3]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub

In [4]:
# Step 2: Pull the ReAct prompt from Langchain Hub
prompt = hub.pull('hwchase17/react') # pulls the standard ReAct agent prompt

d:\2025_Coding\Campusx\Langchain\venv\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [21]:
print(prompt)

input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'} template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'


### DuckDuckSearchRun and simple ReAct prompt gives repetition , so i needed to hardcode my prompt and use DuckDuckGoSearchResults

In [85]:
from langchain_core.prompts import PromptTemplate

prompt_new = PromptTemplate(
    template = """
    You are a helpful AI Travel assistant.
    You have access to the following tools:
    {tools}

    Rules:
    - If you already know the answer from your own knowledge, skip Action/Action Input and go directly to "Final Answer".
    - Only use a tool if you truly lack the information.
    - Action must be EXACTLY one of [{tool_names}] — nothing else.
    - Never invent tool names.
    - Do not repeat the same search or action.
    - Stop once you find enough info to answer.
    - Be concise and complete.


    Use the following format:

    Question: the input question you must answer
    Thought: you should always think about what to do
    Action: the action to take, should be one of [{tool_names}] - nothing else
    Action Input: the input to the action
    Observation: the result of the action
    ... (repeat Thought/Action/Action Input/Observation N times)
    Once you have enough information to fully answer, write:
    Thought: I now know the final answer
    Final Answer: <your complete answer here>
    Do not call any more tools after this point.
    
    Begin!

    Question: {input}
    Thought:{agent_scratchpad}
    """,
    input_variables = ["tools","input","tool_names","agent_scratchpad"]
)


In [86]:
from langchain_community.tools import DuckDuckGoSearchResults

search_tool_new = DuckDuckGoSearchResults(num_results= 6)

tools_new = [search_tool_new]




In [87]:
# Step 3: creating react agent from 
agent = create_react_agent(
    llm = llm,
    tools = tools_new,
    prompt = prompt_new
) 

In [88]:
"""Agent does the planning work, and Agent_Executor will execute each step of the plan"""

'Agent does the planning work, and Agent_Executor will execute each step of the plan'

In [89]:
# step 4 : wrap it with AgentExecutor
agent_executor = AgentExecutor(
    agent = agent,
    tools = tools_new,
    verbose = True,
    max_iterations= 6
)

# as verbose is true , so whatever the agent thinks while making plan will be seen in the console

Key Insights:
1. React Prompt was made like: Though , Action , Observation and repeat again
    i) so create_react_agent function must also get such formatted prompt
    
2. need to give agent sufficient iterations to search 
3. 'agent_scratchpad' is string where we get all the verbose of agent

### Campusx code resumes from here

In [91]:
# Step 5 : Invoke
response = agent_executor.invoke({"input": "best way to reach goa from delhi"})
print(response)



> Entering new AgentExecutor chain...
Question: best way to reach goa from delhi
Thought: I need to find the best route from Delhi to Goa. I should check if there are any direct flights or trains available.

Action: duckduckgo_results_json
Action Input: "Delhi to Goa flights" OR "Delhi to Goa trains"

d:\2025_Coding\Campusx\Langchain\venv\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


snippet: New Delhi is jointly administered by the federal government of India and the local government of Delhi, and serves as the capital of the nation …, title: Delhi - Wikipedia, link: https://en.m.wikipedia.org/wiki/Delhi, snippet: It explores the wonders of Delhi, its heritage, the art and craft, the diverse cuisine and culture. A symbol of the country's rich past and thriving …, title: Welcome to Delhi Tourism: Official Tourism Website for Government of ..., link: https://delhitourism.gov.in/index.html, snippet: Capital of India, the historical and vivacious Delhi is blessed with innumerable monuments, scenic locations and prominent shrines., title: 33 Best Places to Visit in Delhi - Delhi Tourism 2025, link: https://delhitourism.travel/places-to-visit-in-delhi, snippet: 3 days ago · Delhi is of great historical significance as an important commercial, transport, and cultural hub, as well as the …, title: Delhi | History, Population, Map, & Facts | Britannica, link: https://www.b

d:\2025_Coding\Campusx\Langchain\venv\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


snippet: Delhi to Goa Flights - Book Delhi to Goa flight tickets at lowest airfare only at MakeMyTrip. Use promo code MMTDEAL & get Flat 8% Upto …, title: Delhi to Goa Flights, Fares @ ₹3868 + Upto ₹800 OFF, link: https://www.makemytrip.com/flights/new_delhi-goa-cheap-airtickets.html, snippet: Sep 3, 2014 · Book your Delhi to Goa flight at the lowest airfare. Apply promo code "GIRUSH" to enjoy up to 10% off on your booking. For quick …, title: Delhi to Goa Flights, Fares @ ₹3868 + upto ₹1000 OFF - Goibibo, link: https://www.goibibo.com/flights/delhi-to-goa-flights/, snippet: Book your Delhi to Goa flights at the lowest airfare. Apply promo code "YTDELIGHT" to enjoy up to 15% off on your booking. For quick …, title: Delhi to Goa Flight – Book from @ ₹3919 + Upto ₹3,700 OFF, link: https://www.yatra.com/flight-schedule/delhi-to-goa-flights.html, snippet: Compare and find cheap flight tickets from New Delhi (DEL) to Goa (GOI). Book directly with no added fees., title: Flight tickets from I

In [ ]:
response['output']

"There are several ways to reach Goa from Delhi, including:\n\n1. Flight: You can take a direct flight from Delhi's Indira Gandhi International Airport to Goa's Dabolim Airport. Flight duration is approximately 1.5 hours, and frequency of flights is high.\n2. Train: You can take a train from Delhi's Hazrat Nizamuddin Railway Station to Goa's Madgaon Railway Station. Train journey takes around 24-30 hours, depending on the type of train and route.\n3. Road: You can drive from Delhi to Goa via the NH8 highway, which takes around 18-20 hours depending on traffic conditions. Alternatively, you can take a bus from Delhi to Goa, which takes around 24-30 hours."

## Inference
* This playlist is now complete, but langchain is not used to large scaled industrial projects.
* For that i need to learn lang-graph
* Now, I'll learn Fast-api